In [1]:
# 필요한 lib 목록들
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
import os
from time import sleep
import csv

In [2]:
# 본인의 Bitquery api키를 입력
api_key = 'BQY23qSW7EjXld8P25U3UCUkuL7hVpvI' # 태형님 API키
headers = {
    'Content-Type': 'application/json',
    'X-API-KEY': api_key
}

Ticker = "SOL"

## 밑에 4가지 값을 조절가능!
# 시작 날짜와 종료 날짜 설정 (변수 정의) 일자 단위로만 추출 가능. 
# 일주일 단위로 넣어도 되긴할 것이지만 중간에 끊기는 불상사가 발생할 수 있으니 단위를 좀 끊어서 뽑는 것을 권장드림.


start_date = "2020-10-01"
end_date = "2020-12-31"

# start_date = "2021-01-01"
# end_date = "2021-03-31"

# 자신이 뽑을 월에 시작 날과 끝날을 지정해 두면 됩니다.
# start_date = datetime(2020,3, 19)
# end_date = datetime(2020, 12, 31)

# # 날짜 리스트 생성
# date_list = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end_date - start_date).days + 1)]

# 페이지네이션을 위해 초기 offset 설정 # 이 부분은 안 건들이는 게 좋음!
offset = 0
batch_size = 25000

## JSON을 개별로 저장한 후에 CSV로 통합하는 코드 - 안정성을 위해 이 코드 이용

In [3]:
## 서버에서 json 파일 불러오는 코드 ## 

# Bitquery 엔드포인트
url = 'https://graphql.bitquery.io/'

# GraphQL 쿼리 템플릿 설정
query_template = '''
query Myquery($limit: Int!, $offset: Int!, $from: ISO8601DateTime, $till: ISO8601DateTime) {
  solana(network: solana) {
    transfers(
      date: {since: $from, till: $till}
      options: {asc: "block.height", limit: $limit, offset: $offset}
      currency: {is: "SOL"}
      amount: {gteq: 100}
    ) {
      receiver {
        address
      }
      sender {
        address
      }
      transaction {
        fee
      }
      block {
        hash
        height
        timestamp {
          time(format: "%Y-%m-%d %H:%M:%S")
        }
      }
      amount: amount(currency: {is: "SOL"})
    }
  }
}
'''

# 시작 날짜와 종료 날짜, 한 json에 몇 개의 데이터가 들어가는지(최대 25,000개) 조정 가능
params = {
    "limit": batch_size, # 위에서 조정 가능
    "offset": offset,
    "from": start_date, # 위에서 조정 가능 
    "till": end_date # 위에서 조정 가능
}

# JSON 파일 저장 디렉토리 설정
output_dir = f'{Ticker}_{start_date}_{end_date}_json_pages'
os.makedirs(output_dir, exist_ok=True)

page_number = 1
empty_count = 0
while True:
    response = requests.post(url, headers=headers, json={'query': query_template, 'variables': params})

    if response.status_code == 200: # 문제 없을 경우 데이터가 추출
        data = response.json()
        if 'data' in data and 'solana' in data['data'] and 'transfers' in data['data']['solana']:
            transfers = data['data']['solana']['transfers']
            if not transfers:
                params["offset"] += params["limit"]  # 다음 페이지로 이동
                print(f"page {page_number} is empty! pass the next page~")
                page_number += 1
                empty_count += 1 # page가 연속으로 지정한 횟수만큼 없으면 코드를 멈춤
                if empty_count >= 5:
                    print("5 consecutive empty pages found. Stopping the process.")
                    break
                sleep(1)
                continue

            # JSON 파일로 저장
            with open(os.path.join(output_dir, f'{Ticker}_{start_date}_{end_date}_page_{page_number}.json'), 'w') as json_file:
                json.dump(transfers, json_file, indent=4)
            
            params["offset"] += params["limit"]  # 다음 페이지로 이동
            print(f"page {page_number} is complete!")
            page_number += 1
            empty_count = 0
        else:
            break  # 데이터가 없으면 종료
    else:
        print(f'Query failed with status code {response.status_code}')
        print(response.text)
        break
    sleep(1)

print(f'Data successfully saved to JSON files in {output_dir}')

page 1 is complete!
page 2 is complete!
page 3 is complete!
page 4 is complete!
page 5 is complete!
page 6 is complete!
page 7 is complete!
page 8 is complete!
page 9 is complete!
page 10 is complete!
page 11 is complete!
page 12 is complete!
page 13 is complete!
page 14 is complete!
page 15 is complete!
page 16 is complete!
page 17 is complete!
page 18 is complete!
page 19 is complete!
page 20 is complete!
page 21 is complete!
page 22 is complete!
page 23 is complete!
page 24 is complete!
page 25 is complete!
page 26 is complete!
page 27 is complete!
page 28 is complete!
page 29 is complete!
page 30 is complete!
page 31 is complete!
page 32 is complete!
page 33 is complete!
page 34 is complete!
page 35 is complete!
page 36 is complete!
page 37 is complete!
page 38 is complete!
page 39 is complete!
page 40 is complete!
page 41 is complete!
page 42 is complete!
page 43 is complete!
page 44 is complete!
page 45 is complete!
page 46 is complete!
page 47 is complete!
page 48 is complete!
p

In [4]:
# JSON 파일 저장 디렉토리 설정
input_dir = f'{Ticker}_{start_date}_{end_date}_json_pages'
output_csv = f'{Ticker}_{start_date}_{end_date}_TRX.csv'

# CSV 파일에 저장할 필드 이름 설정
fieldnames = [
    'timestamp', 'block_height', 'sender_address', 
    'receiver_address', 'amount', 'trx_hash', 'trx_gasPrice'
]

# 모든 JSON 데이터 리스트에 저장
all_transfers = []

# JSON 파일들 읽기
for filename in os.listdir(input_dir):
    if filename.endswith('.json'):
        with open(os.path.join(input_dir, filename), 'r') as jsonfile:
            transfers = json.load(jsonfile)

            # JSON 데이터에서 필요한 필드 추출하여 리스트에 추가
            for transfer in transfers:
                row = {
                    'timestamp': transfer['block']['timestamp']['time'],
                    'block_height': transfer['block']['height'],
                    'sender_address': transfer['sender']['address'],
                    'receiver_address': transfer['receiver']['address'],
                    'amount': transfer['amount'],
                    'trx_hash': transfer['block']['hash'],
                    'trx_gasPrice': transfer['transaction']['fee']
                }
                all_transfers.append(row)

# 타임스탬프를 기준으로 정렬
all_transfers_sorted = sorted(all_transfers, key=lambda x: datetime.strptime(x['timestamp'], '%Y-%m-%d %H:%M:%S'))

# CSV 파일 생성 및 필드 이름 작성
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # 정렬된 데이터 CSV 파일에 작성
    for row in all_transfers_sorted:
        writer.writerow(row)

print(f'Data successfully saved to CSV file {output_csv}')

Data successfully saved to CSV file SOL_2020-10-01_2020-12-31_TRX.csv


2020년 뽑는중\
3월19~6월30까지 뽑는중 9.9 10:30
전체 뽑는 것 완료. 9.10 11:24

## For문 써서 전체 한 번에 하는 코드

In [4]:
## 서버에서 json 파일 불러오는 코드 ## 

# Bitquery 엔드포인트
url = 'https://graphql.bitquery.io/'

# GraphQL 쿼리 템플릿 설정
query_template = '''
query Myquery($limit: Int!, $offset: Int!, $from: ISO8601DateTime, $till: ISO8601DateTime) {
  solana(network: solana) {
    transfers(
      date: {since: $from, till: $till}
      options: {asc: "block.height", limit: $limit, offset: $offset}
      currency: {is: "SOL"}
      amount: {gteq: 100}
    ) {
      receiver {
        address
      }
      sender {
        address
      }
      transaction {
        fee
      }
      block {
        hash
        height
        timestamp {
          time(format: "%Y-%m-%d %H:%M:%S")
        }
      }
      amount: amount(currency: {is: "SOL"})
    }
  }
}

'''

for num in range(len(date_list)):
    # 시작 날짜와 종료 날짜, 한 json에 몇 개의 데이터가 들어가는지(최대 25,000개) 조정 가능
    params = {
        "limit": batch_size, # 위에서 조정 가능
        "offset": offset,
        "from": date_list[num], # 위에서 조정 가능 
        "till": date_list[num] # 위에서 조정 가능
    }
    
    # JSON 파일 저장 디렉토리 설정
    output_dir = f'{Ticker}_{date_list[num]}_json_pages'
    os.makedirs(output_dir, exist_ok=True)
    
    page_number = 1
    empty_count = 0
    while True:
        response = requests.post(url, headers=headers, json={'query': query_template, 'variables': params})
    
        if response.status_code == 200: # 문제 없을 경우 데이터가 추출
            data = response.json()
            if 'data' in data and 'solana' in data['data'] and 'transfers' in data['data']['solana']:
                transfers = data['data']['solana']['transfers']
                if not transfers:
                    params["offset"] += params["limit"]  # 다음 페이지로 이동
                    print(f"page {page_number} is empty! pass the next page~")
                    page_number += 1
                    empty_count += 1 # page가 연속으로 몇번 없으면 코드를 멈춤
                    if empty_count >= 5:
                        print("5 consecutive empty pages found. Stopping the process.")
                        break
                    sleep(1)
                    continue
    
                # JSON 파일로 저장
                with open(os.path.join(output_dir, f'{Ticker}_{date_list[num]}_page_{page_number}.json'), 'w') as json_file:
                    json.dump(transfers, json_file, indent=4)
                
                params["offset"] += params["limit"]  # 다음 페이지로 이동
                print(f"page {page_number} is complete!")
                page_number += 1
                empty_count = 0
            else:
                break  # 데이터가 없으면 종료
        else:
            print(f'Query failed with status code {response.status_code}')
            print(response.text)
            break
        sleep(3)
    
    print(f'Data successfully saved to JSON files in {output_dir}')
    
    # JSON 파일 저장 디렉토리 설정
    input_dir = f'{Ticker}_{date_list[num]}_json_pages'
    output_csv = f'{Ticker}_{date_list[num]}_TRX.csv'
    
    # CSV 파일에 저장할 필드 이름 설정
    fieldnames = [
        'timestamp', 'block_height', 'sender_address', 
        'receiver_address', 'amount', 'trx_hash', 'trx_gasPrice'
    ]
    
    # 모든 JSON 데이터 리스트에 저장
    all_transfers = []
    
    # JSON 파일들 읽기
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            with open(os.path.join(input_dir, filename), 'r') as jsonfile:
                transfers = json.load(jsonfile)
    
                # JSON 데이터에서 필요한 필드 추출하여 리스트에 추가
                for transfer in transfers:
                    row = {
                        'timestamp': transfer['block']['timestamp']['time'],
                        'block_height': transfer['block']['height'],
                        'sender_address': transfer['sender']['address'],
                        'receiver_address': transfer['receiver']['address'],
                        'amount': transfer['amount'],
                        'trx_hash': transfer['block']['hash'],
                        'trx_gasPrice': transfer['transaction']['fee']
                    }
                    all_transfers.append(row)
    
    # 타임스탬프를 기준으로 정렬
    all_transfers_sorted = sorted(all_transfers, key=lambda x: datetime.strptime(x['timestamp'], '%Y-%m-%d %H:%M:%S'))
    
    # CSV 파일 생성 및 필드 이름 작성
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        # 정렬된 데이터 CSV 파일에 작성
        for row in all_transfers_sorted:
            writer.writerow(row)
    
    print(f'Data successfully saved to CSV file {output_csv}')
    print('----------------------------------------------------')

page 1 is empty! pass the next page~
page 2 is empty! pass the next page~
page 3 is empty! pass the next page~
page 4 is empty! pass the next page~
page 5 is empty! pass the next page~
5 consecutive empty pages found. Stopping the process.
Data successfully saved to JSON files in SOL_2020-03-19_json_pages
Data successfully saved to CSV file SOL_2020-03-19_TRX.csv
----------------------------------------------------
page 1 is empty! pass the next page~
page 2 is empty! pass the next page~
page 3 is empty! pass the next page~
page 4 is empty! pass the next page~
page 5 is empty! pass the next page~
5 consecutive empty pages found. Stopping the process.
Data successfully saved to JSON files in SOL_2020-03-20_json_pages
Data successfully saved to CSV file SOL_2020-03-20_TRX.csv
----------------------------------------------------
page 1 is empty! pass the next page~
page 2 is empty! pass the next page~
page 3 is empty! pass the next page~
page 4 is empty! pass the next page~
Query failed w

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))